# Imports and Preprocessing

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasRegressor

In [3]:
import os
import pandas as pd
folder_path = 'Data_Test_v1'
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

# Demand Forecasting with ANN Model

In [4]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from scipy.stats import reciprocal

# Define function to create the ANN model 
def create_model(n_hidden, n_neurons, learning_rate, activation, input_shape=[3]): 
    model = Sequential()
    model.add(Dense(n_neurons, activation="relu", input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(1))
    optimizer = Adam() #learning_rate=learning_rate_input
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

# Define function to build the model
def model_builder(n_hidden=1, n_neurons=30, learning_rate=3e-3 , activation = 'relu'):
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, learning_rate=learning_rate, activation=activation) 

# Define the hyperparameters to search for the Randomized Search
param_distribs = {
    "n_hidden": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10, 20, 30, 40, 50],
    "activation": ['relu', 'sigmoid', 'tanh']
}

# Create the model and optimize with Randomized Search
model_ANN = model_builder()
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')

# Fit the model on the training data
rnd_search_cv_ANN.fit(data_train, target_train)
print(rnd_search_cv_ANN.best_params_)

# Predict the target values
target_pred_ANN = rnd_search_cv_ANN.predict(data_test)
train_pred_ANN = rnd_search_cv_ANN.predict(data_train)

mse = mean_squared_error(target_test, target_pred_ANN)
print("MSE:", mse)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 32, 'epochs': 50, 'learning_rate': 0.00032199876460927286, 'n_hidden': 1, 'n_neurons': 89}
MSE: 120693.29315258816


# Demand Forecasting with DT (LightGBM)

In [5]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Create the model and define the hyperparameters to search for the Randomized Search
model_DT = lgb.LGBMRegressor()
param_distribs = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

# Create the Randomized Search object
rnd_search_cv_DT = RandomizedSearchCV(model_DT, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')

# Fit the model on the training data
rnd_search_cv_DT.fit(data_train, target_train)

# Get the best parameters and best estimator
best_params = rnd_search_cv_DT.best_params_
best_estimator = rnd_search_cv_DT.best_estimator_

# Make predictions
target_pred_DT = best_estimator.predict(data_test)
train_pred_DT = best_estimator.predict(data_train)

# Calculate the MSE
mse = mean_squared_error(target_test, target_pred_DT)
print("MSE:", mse)



[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49
[LightGBM] [Info] Number of data points in the train set: 138, number of used features: 3
[LightGBM] [Info] Start training from score 1519.210145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

# Solving Order Allocation Problem (Sample Average Approximation)

In [6]:
# Load the supplier data
df_suppliers = pd.read_csv('Data_Test_v1\Supplier_Data.csv', sep=';')
print(df_suppliers.head())

   supplier  capacity  price
0         1       350      5
1         2       350      7
2         3       450      8
3         4       260      7
4         5       170      7


In [7]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_ANN_flatten = train_pred_ANN.flatten()
forecast_error_ANN = target_train_flatten - train_pred_ANN_flatten

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = [target_pred_ANN[i] + np.random.choice(forecast_error_ANN) for _ in range(10)]

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the order allocation problem
    for demand in demand_scenarios:
        # Define the order allocation problem
        prob = LpProblem("Order_Allocation", LpMinimize)

        # Define the decision variables
        x = LpVariable.dicts("x", df_suppliers.index, lowBound=0, cat='Continuous')

        # Define the objective function
        prob += lpSum([df_suppliers.loc[i, 'price'] * x[i] for i in df_suppliers.index])
        
        # Define the constraints
        for i in df_suppliers.index:
            prob += x[i] <= df_suppliers.loc[i, 'capacity']
        prob += lpSum([x[i] for i in df_suppliers.index]) == demand  # Total quantity ordered must meet the demand

        # Solve the problem
        prob.solve()

        # Store the solution for this scenario
        saa_solutions.append([v.varValue for v in prob.variables()])

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_ANN.append(final_allocation)



In [10]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

# Initialize an empty list to store the final order quantities
final_order_quantities_DT = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_DT_flatten = train_pred_DT.flatten()
forecast_error_DT = target_train_flatten - train_pred_DT_flatten

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = [target_pred_DT[i] + np.random.choice(forecast_error_DT) for _ in range(10)]

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the order allocation problem
    for demand in demand_scenarios:
        # Define the order allocation problem
        prob = LpProblem("Order_Allocation", LpMinimize)

        # Define the decision variables
        x = LpVariable.dicts("x", df_suppliers.index, lowBound=0, cat='Continuous')

        # Define the objective function
        prob += lpSum([df_suppliers.loc[i, 'price'] * x[i] for i in df_suppliers.index])
        
        # Define the constraints
        for i in df_suppliers.index:
            prob += x[i] <= df_suppliers.loc[i, 'capacity']
        prob += lpSum([x[i] for i in df_suppliers.index]) == demand  # Total quantity ordered must meet the demand

        # Solve the problem
        prob.solve()

        # Store the solution for this scenario
        saa_solutions.append([v.varValue for v in prob.variables()])

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_DT.append(final_allocation)

# # Solving Order Allocation Problem (Model Based)

In [ ]:
# Load the supplier data
df_suppliers = pd.read_csv('Data_Test_v1\Supplier_Data.csv', sep=';')
print(df_suppliers.head())

In [ ]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN_model = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_ANN_flatten = train_pred_ANN.flatten()
forecast_error_ANN = target_train_flatten - train_pred_ANN_flatten
# Estimate the standard deviation of forecast errors
std_dev_ANN = np.std(forecast_error_ANN, ddof=1)  # ddof=1 provides an unbiased estimator by dividing by N-1

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = target_pred_ANN[i] + np.random.normal(loc=0, scale=std_dev_ANN, size=100)

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the order allocation problem
    for demand in demand_scenarios:
        # Define the order allocation problem
        prob = LpProblem("Order_Allocation", LpMinimize)

        # Define the decision variables
        x = LpVariable.dicts("x", df_suppliers.index, lowBound=0, cat='Continuous')

        # Define the objective function
        prob += lpSum([df_suppliers.loc[i, 'price'] * x[i] for i in df_suppliers.index])
        
        # Define the constraints
        for i in df_suppliers.index:
            prob += x[i] <= df_suppliers.loc[i, 'capacity']
        prob += lpSum([x[i] for i in df_suppliers.index]) == demand  # Total quantity ordered must meet the demand

        # Solve the problem
        prob.solve()

        # Store the solution for this scenario
        saa_solutions.append([v.varValue for v in prob.variables()])

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_ANN_model.append(final_allocation)


In [ ]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

# Initialize an empty list to store the final order quantities
final_order_quantities_DT_model = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_DT_flatten = train_pred_DT.flatten()
forecast_error_DT = target_train_flatten - train_pred_DT_flatten
# Estimate the standard deviation of forecast errors
std_dev_DT = np.std(forecast_error_DT, ddof=1)  # ddof=1 provides an unbiased estimator by dividing by N-1

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = target_pred_DT[i] + np.random.normal(loc=0, scale=std_dev_DT, size=100)

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the order allocation problem
    for demand in demand_scenarios:
        # Define the order allocation problem
        prob = LpProblem("Order_Allocation", LpMinimize)

        # Define the decision variables
        x = LpVariable.dicts("x", df_suppliers.index, lowBound=0, cat='Continuous')

        # Define the objective function
        prob += lpSum([df_suppliers.loc[i, 'price'] * x[i] for i in df_suppliers.index])
        
        # Define the constraints
        for i in df_suppliers.index:
            prob += x[i] <= df_suppliers.loc[i, 'capacity']
        prob += lpSum([x[i] for i in df_suppliers.index]) == demand  # Total quantity ordered must meet the demand

        # Solve the problem
        prob.solve()

        # Store the solution for this scenario
        saa_solutions.append([v.varValue for v in prob.variables()])

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_DT_model.append(final_allocation)

# Analysis of Results

In [13]:
# Convert final order quantities to a numpy array
final_order_quantities_ANN = np.array(final_order_quantities_ANN)
final_order_quantities_DT = np.array(final_order_quantities_DT)

# Calculate the total cost of the orders
total_cost_ANN = np.sum(final_order_quantities_ANN * df_suppliers['price'].values)
total_cost_DT = np.sum(final_order_quantities_DT * df_suppliers['price'].values)

# Calculate the total demand
total_demand = np.sum(target_test.values.flatten())
total_pred_demand_ANN = np.sum(target_pred_ANN)
total_pred_demand_DT = np.sum(target_pred_DT)

# Calculate the total unmet demand
total_unmet_demand_ANN = total_demand - np.sum(final_order_quantities_ANN)
total_unmet_demand_DT = total_demand - np.sum(final_order_quantities_DT)
predict_unmet_demand_ANN = total_pred_demand_ANN - np.sum(final_order_quantities_ANN)
predict_unmet_demand_DT = total_pred_demand_DT - np.sum(final_order_quantities_DT)

print(f"Total cost of orders ANN: {total_cost_ANN}")
print(f"Total cost of orders DT: {total_cost_DT}")
print(f"Total demand: {total_demand}")
print(f"Total unmet demand ANN: {total_unmet_demand_ANN}")
print(f"Total unmet demand DT: {total_unmet_demand_DT}")
print(f"Total mismatch with prediction ANN: {predict_unmet_demand_ANN}")
print(f"Total mismatch with prediction DT: {predict_unmet_demand_DT}")

Total cost of orders ANN: 466148.45353955403
Total cost of orders DT: 475047.68024919997
Total demand: 82292
Total unmet demand ANN: 905.6420306579967
Total unmet demand DT: -440.9896635999903
Total mismatch with prediction ANN: 247.68890565799666
Total mismatch with prediction DT: 234.51346630997432
